In [9]:
%matplotlib inline
from __future__ import print_function

import numpy as np
import keras
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.layers.core import Dense, Flatten, Dropout, Activation
from keras.layers import GlobalAveragePooling2D, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers.advanced_activations import PReLU


In [10]:
train_path = 'data/train'
valid_path = 'data/valid'
batch_size = 32
target_size = 224

train_batches = ImageDataGenerator(rescale=1./ 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True).flow_from_directory(train_path, target_size=(target_size, target_size), batch_size=batch_size, class_mode='categorical')
valid_batches = ImageDataGenerator(rescale=1./ 255).flow_from_directory(valid_path, target_size=(target_size,target_size), batch_size=batch_size, class_mode='categorical')

Found 6849 images belonging to 120 classes.
Found 3373 images belonging to 120 classes.


In [11]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(target_size,target_size,3))
#base_model = Xception(weights='imagenet', include_top=False, input_shape=(target_size, target_size, 3))

In [4]:
for layer in base_model.layers:
    layer.trainable = False

In [7]:
output = base_model.output
output = BatchNormalization()(output)
output = GlobalAveragePooling2D()(output)
output = Dropout(0.5)(output)
output = Dense(1024, activation='relu')(output)
output = Dropout(0.5)(output)
predictions = Dense(120, activation='softmax', name='predictions')(output)

In [8]:
output = base_model.output
output = GlobalAveragePooling2D()(output)
output = Dense(1024, activation='relu')(output)
output = Dropout(0.3)
output = Dense(1024, activation='relu')(output)
predictions = Dense(120, activation='softmax', name='predictions')(output)

ValueError: Layer dense_6 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.layers.core.Dropout'>. Full input: [<keras.layers.core.Dropout object at 0x7f2a92b16da0>]. All inputs to the layer should be tensors.

In [ ]:
#output = base_model.get_layer(index=-1).output
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9)

In [ ]:
#model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=train_batches.n//batch_size, validation_data=valid_batches, validation_steps=valid_batches.n//batch_size, epochs=50)